# Trump Tweet Ananlysis
## Table of content
* 1.Reading csv
* 2.Preprocessing
    * 2.1 Extracting the year,month,dates,hour,minute,second from date coloumn
    * 2.2 Removing tag names from tweets
    * 2.3 preprocessing of tweet text
* 3.Most active hour on twitter
* 4.Number of tweet 
    * 4.1 Emotional Analysis over the years
* 5.Average number of tweet in a day
* 6.Most retweeted tweet
* 7.Most Like tweets
* 8.Most liked tweet durning Presidential year
* 9.Most retweeted tweet durning presidential year
* 10.Word priority
  * 10.1 Business year
  * 10.2 compain year
  * 10.3 presidential year

### 1. Reading csv

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
import numpy as np

#ignore warning messages
import warnings
warnings.filterwarnings('ignore')
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objects as go
import plotly.express as px
import csv

from wordcloud import WordCloud


df= pd.read_csv("trump-tweets/trumptweets.csv")

In [2]:
colorblind = ['#67001f',
              '#a50026', 
              '#d73027', 
              '#f46d43', 
              '#fdae61', 
              '#fee090', 
              '#ffffbf',
              '#e0f3f8', 
              '#abd9e9', 
              '#74add1', 
              '#4575b4', 
              '#313695',
              '#053061'
             ]

In [3]:
df.head()

,id,link,content,date,retweets,favorites,mentions,hashtags,geo
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 20:54:25,500,868,NaN,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-05 03:00:10,33,273,NaN,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 15:38:08,12,18,NaN,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 22:40:15,11,24,NaN,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 16:07:28,1399,1965,NaN,NaN,NaN


In [4]:
df.drop(['id','link'],axis=1,inplace=True)
df.shape

(41122, 7)

In [5]:
df.head()

,content,date,retweets,favorites,mentions,hashtags,geo
0,Be sure to tune in and watch Donald Trump on L...,2009-05-04 20:54:25,500,868,NaN,NaN,NaN
1,Donald Trump will be appearing on The View tom...,2009-05-05 03:00:10,33,273,NaN,NaN,NaN
2,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 15:38:08,12,18,NaN,NaN,NaN
3,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 22:40:15,11,24,NaN,NaN,NaN
4,"""My persona will never be that of a wallflower...",2009-05-12 16:07:28,1399,1965,NaN,NaN,NaN


### 2. Preprocessing

#### 2.1 Extracting the year,month,dates,hour,minute,second from date coloumn

In [6]:
year=[]
month=[]
date=[]
hour=[]
minute=[]
second=[]
for x in df['date']:
    year.append(int(x.split("-")[0]))
    month.append(int(x.split("-")[1]))
    date.append(int(x.split("-")[2].split(" ")[0]))
    hour.append(int(x.split("-")[2].split(" ")[1].split(":")[0]))
    minute.append(int(x.split("-")[2].split(" ")[1].split(":")[1]))
    second.append(int(x.split("-")[2].split(" ")[1].split(":")[2]))

df['year']=year
df['month']=month
df['dates']=date
df['hour']=hour
df['minute']=minute
df['second']=second
df.drop(['date'],axis=1,inplace=True)

#### 2.2. Removing tag names from tweets

In [7]:
df.head()

,content,retweets,favorites,mentions,hashtags,geo,year,month,dates,hour,minute,second
0,Be sure to tune in and watch Donald Trump on L...,500,868,NaN,NaN,NaN,2009,5,4,20,54,25
1,Donald Trump will be appearing on The View tom...,33,273,NaN,NaN,NaN,2009,5,5,3,0,10
2,Donald Trump reads Top Ten Financial Tips on L...,12,18,NaN,NaN,NaN,2009,5,8,15,38,8
3,New Blog Post: Celebrity Apprentice Finale and...,11,24,NaN,NaN,NaN,2009,5,8,22,40,15
4,"""My persona will never be that of a wallflower...",1399,1965,NaN,NaN,NaN,2009,5,12,16,7,28


In [8]:
#[ ] \@\S+ sitassi per togliere i tag
import re
content=[]
for tweet in df["content"]:
    content.append(re.sub('\@\S+','',tweet))

#### 2.3 preprocessing of tweet text
* remove url

In [9]:
# loading stop words from nltk library
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
def preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        #Removing link
        url_pattern = r'((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?'
        total_text = re.sub(url_pattern, ' ', total_text)
        # replace every special char with space
        #total_text = re.sub('[^a-zA-Z0-9\n]', ' ', total_text)
        # replace multiple spaces with single space
        total_text = re.sub('\s+',' ', total_text)
        #total_text=total_text.replace('realdonaldtrump','').replace('donald','').replace('trump','')
        # converting all the chars into lower-case.
        total_text = total_text.lower()
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from the data
            if not word in stop_words:
                word=(word)
                string += word + " "
        
        df[column][index] = string

for index, row in df.iterrows():
    if type(row['content']) is str:
        preprocessing(row['content'], index, 'content')

In [10]:
df.head()

,content,retweets,favorites,mentions,hashtags,geo,year,month,dates,hour,minute,second
0,sure tune watch donald trump late night david ...,500,868,NaN,NaN,NaN,2009,5,4,20,54,25
1,donald trump appearing view tomorrow morning d...,33,273,NaN,NaN,NaN,2009,5,5,3,0,10
2,donald trump reads top ten financial tips late...,12,18,NaN,NaN,NaN,2009,5,8,15,38,8
3,new blog post: celebrity apprentice finale les...,11,24,NaN,NaN,NaN,2009,5,8,22,40,15
4,"""my persona never wallflower - i’d rather buil...",1399,1965,NaN,NaN,NaN,2009,5,12,16,7,28


### 3. Most active hour on twitter

In [11]:
Category=df['hour'].value_counts().sort_index()

fig = px.pie(
    df,
    values = Category.values,
    labels = Category.index,
    #title = "Most active hour on Twitter",
    color_discrete_sequence = colorblind,
    #names = Category.index   
)
fig.update_traces(textposition='inside', textinfo='label+percent')

fig.show()

In [13]:
import plotly.io as pio
img = fig.write_image("fig1.png", width=600, height=350, scale=2)

In [14]:
hour = df['hour'].value_counts().reset_index(name='counts')

colors = ['#92c5de',] * 24
colors[0] = '#053061'

fig = go.Figure(data=[go.Bar(
    x=hour['index'],
    y=hour['counts'],
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout()

### 4. Number of tweet 

In [15]:
year = df['year'].value_counts().reset_index(name='counts')

colors = ['#92c5de',] * 12
colors[0] = '#053061'

fig = go.Figure(data=[go.Bar(
    x=year['index'],
    y=year['counts'],
    marker_color=colors, # marker color can be a single color value or an iterable
)])
fig.update_layout(title_text="Numbers vs Years")

**OSSEVAZIONI** - Trump risulta molto attivo nel 2013 in quanto commenta ogni decisione presa da obama sul tema iran. Dal paragrafo 7 infatti notiamo come uno dei suoi tweet preferiti sia proprio un commento a questi fatti. Essendo lui repubblicano (al contrario di obama democratico) raccoglieva il consenso di tutta la fetta di popolazione che non si sentiva rappresentata dalle decisioni dell’allora presidente. Obama ormai non poteva essere rieletto, ma comunque spoiler not spoiler ha vinto alle elezioni successive a sorpresa contro Hillary Clinton(2017-2021)

#### 4.1 Sentiment Analysis of tweet

In [16]:
import text2emotion as te

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sarac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sarac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sarac\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
contents = df['content']
years = df['year']
list_years = years.tolist()
list_contents = contents.tolist()

#creo 2 dizionari per cui per ogni anno inserisco lista di testi
dict_contents = {}
dict_emotions_x_years = {}
for y in years:
    dict_contents[y] = []
    e = {
        'joy': 0,
        'anger': 0,
        'surprise': 0,
        'sadness': 0,
        'fear': 0
    }
    dict_emotions_x_years[y] = e

print(dict_contents, dict_emotions_x_years)

{2009: [], 2010: [], 2011: [], 2012: [], 2013: [], 2014: [], 2015: [], 2016: [], 2017: [], 2018: [], 2019: [], 2020: []} {2009: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2010: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2011: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2012: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2013: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2014: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2015: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2016: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2017: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2018: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2019: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}, 2020: {'joy': 0, 'anger': 0, 'surprise': 0, 'sadness': 0, 'fear': 0}}


In [18]:
#riempio i dizionari

for i in range(len(list_years)):
    content = list_contents[i]
    year = list_years[i]
    list_from_dict = dict_contents[year]
    list_from_dict.append(content)


In [19]:
#funzione che mi permette di calcolare quale è l'emozione prevalente di un testo
# somma questa emozione al dizionario corretto

def analyze_emotion(emotion_dict, dict_emotions, y):
    max_value = max(emotion_dict, key=emotion_dict.get)
    sub_dict = dict_emotions_x_years.get(y)
    
    if max_value  == 'Happy':
        sub_dict['joy'] = sub_dict.get('joy') + 1
        dict_emotions[y] = sub_dict 
    elif max_value  == 'Angry':
        sub_dict['anger'] = sub_dict.get('anger') + 1
        dict_emotions[y] = sub_dict 
    elif max_value  == 'Surprise':
        sub_dict['surprise'] = sub_dict.get('surprise') + 1
        dict_emotions[y] = sub_dict 
    elif max_value  == 'Sad':
        sub_dict['sadness'] = sub_dict.get('sadness') + 1
        dict_emotions[y] = sub_dict 
    else:
        sub_dict['fear'] = sub_dict.get('fear') + 1
        dict_emotions[y] = sub_dict 
    
    return dict_emotions

In [ ]:
for y in dict_contents:
    for c in dict_contents[y]:
        emotion_dict = te.get_emotion(c)
        dict_emotions_x_years = analyze_emotion(emotion_dict, dict_emotions_x_years, y)

In [ ]:
#trasforma il dizionario in una lista di liste per salvarlo in un csv

list_from_dict = []

for y in dict_emotions_x_years:
    sub_dict = dict_emotions_x_years[y]
    inner_list = [y, sub_dict['joy'], sub_dict['anger'], sub_dict['surprise'], sub_dict['sadness'], sub_dict['fear']]
    list_from_dict.append(inner_list)

In [ ]:
#salvataggio nel csv

with open('emotions_x_years.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(list_from_dict)

#### 4.2 Visualizzazione della sentiment analysis

In [ ]:
df_emotions = pd.read_csv("emotions_x_years.csv")
df_emotions.columns = ['year','joy', 'anger', 'surprise', 'sadness', 'fear']
df_emotions.head()

In [ ]:
fig = px.line(df_emotions, x="year", y=df_emotions.columns[1:],
              title='Emotions Over the Years')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

### 5. Average number of tweet in a day per year

In [ ]:
import plotly.graph_objects as go
    
Category=(df['year'].value_counts()/365).sort_index()
Category
x=Category.index
y=Category.values

layout = go.Layout(
   paper_bgcolor='rgba(0,0,0,0)',
   plot_bgcolor='rgba(0,0,0,0)'
)
fig = go.Figure(layout=layout)
fig.update_layout(yaxis={'visible': False, 'showticklabels': False})
 
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
 
fig.add_trace(
   go.Scatter(
       x=x,
       y=p(x),
       mode='lines',
       name='trend',
       marker=dict(color='#b2182b')
   ))
 
y_arrotondati=[]

for n in y:
    if (n>0 and n<1) :
        y_arrotondati.append(round(n,1))
    else:
        y_arrotondati.append(int(n))

fig.add_trace(
   go.Bar(
       x=x,
       y=y,
       name='media tweet x giorno',
       text = y_arrotondati,
       marker=dict(color='#d1e5f0')
   ))
 
fig.show()

### 6. Most retweeted tweet

In [ ]:
for index, value in df['retweets'].nlargest(3).items():
    print('contenuto: ' , df.iloc[index]['content'], '|| anno: ', df.iloc[index]['year'])

### 7. Most Like tweets 

In [ ]:
for index, value in df['favorites'].nlargest(3).items():
    print('contenuto: ' ,df.iloc[index]['content'], '|| anno: ', df.iloc[index]['year'])


### 8. Most tag names from tweets

In [ ]:
#[ ] barchart orizzontale - ruotare nomi - esaltare differenze (es.tra 12 e 13)

from collections import Counter
import re

mention_df=df.dropna(subset=["mentions"])
mentions=[]

for x in mention_df["mentions"]:
    x=x.replace("@","")
    #x=re.sub(r'[\s]+',' ',)
    x=x.strip()
    if not x.strip()=="":
        mentions.append(x)

Top_ten_mentions=Counter(mentions).most_common(10)
#print(Top_ten_mentions)

name=[]
number=[]

for x in Top_ten_mentions:
    name.append(x[0])
    number.append(x[1])

fig = go.Figure(
    data=[
        go.Bar(x=number, y=name)])

fig.update_traces(
    orientation='h',
    text=number,
    marker_color='#4575b4', 
    opacity=0.8)

fig.update_layout(
    plot_bgcolor='#ffffff',
    title=go.layout.Title(
       text="Number x Mention<br><sup>10 most mentioned accounts</sup>",
       xref="paper",
       x=0
   ))
fig.show()

### 9. Most liked tweet durning Presidential year

In [ ]:
president_date=president_year=df[((df['year']>=2017) &(df['month']>1))]
print(df.iloc[president_date['favorites'].idxmax()]['content'])
print(df.iloc[president_date['favorites'].idxmax()]['year'])

### 10. Most retweeted tweet durning presidential year

In [ ]:
president_date=president_year=df[((df['year']>=2017) &(df['month']>1))]
print(df.iloc[president_date['retweets'].idxmax()]['content'])
print(df.iloc[president_date['retweets'].idxmax()]['year'])

### 11. Word priority

#### 11.1 Durning Business Year - TREE MAPS

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import squarify

business_year=df[(df['year']<=2017)]

#trasforma parole in vettori
vec = CountVectorizer().fit(business_year['content'])

#trasforma vettori in matrici
bag_of_words = vec.transform(business_year['content'])

#somma i valori nelle colonne 
sum_words = bag_of_words.sum(axis=0) 

# vec.vocabulary_.items() --> dict delle parole utilizzate
#sum_words[0, idx] --> prende valore dalla colonna 0 scorrendo per idx
words_freq = [
    (word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()
]

words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

y = dict(words_freq[:30])

# disegna il quadrato
fig = plt.figure(figsize=(15, 15))

squarify.plot(
    sizes = y.values(), #dimensione quadrato dipende dalla freq
    label = y.keys(), 
    color=sns.color_palette(colorblind, n_colors=10),
    linewidth=4, 
    text_kwargs={'fontsize':14, 'fontweight' : 'bold'})

plt.title('Top 30 words', position=(0.5, 1.0+0.03), fontsize = 20, fontweight='bold')
plt.axis('off')
plt.show()

#### 11.2 Durning election campaign - TREE MAPS

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import squarify

compain_year=df[(df['year'] == 2016) & (df['month'] >5) | (df['year'] == 2017) &(df['month']==1)]

vec = CountVectorizer().fit(compain_year['content'])
bag_of_words = vec.transform(compain_year['content'])
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

y =dict(words_freq[:30])

colorblind = ['#a50026', '#d73027', '#f46d43', '#fdae61', '#fee090', '#e0f3f8', '#abd9e9', '#74add1', '#4575b4', '#313695']

fig = plt.figure(figsize=(15, 15))
squarify.plot(
    sizes = y.values(), 
    label = y.keys(), 
    color=sns.color_palette(colorblind, n_colors=20),
    linewidth=4, 
    text_kwargs={'fontsize':14, 'fontweight' : 'bold'}
)

plt.title('Top 30 words', position=(0.5, 1.0+0.03), fontsize = 20, fontweight='bold')
plt.axis('off')
plt.show()

#### 11.3 Durning presidentail year

In [ ]:
president_year=df[((df['year']>=2017) &(df['month']>1))]

from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer().fit(president_year['content'])
bag_of_words = vec.transform(president_year['content'])
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
import squarify

y =dict(words_freq[:30])

colorblind = ['#a50026', '#d73027', '#f46d43', '#fdae61', '#fee090', '#e0f3f8', '#abd9e9', '#74add1', '#4575b4', '#313695']

fig = plt.figure(figsize=(15, 15))
squarify.plot(
    sizes = y.values(), label = y.keys(), color=sns.color_palette(colorblind, n_colors=20),
             linewidth=4, text_kwargs={'fontsize':14, 'fontweight' : 'bold'})
plt.title('Top 30 words', position=(0.5, 1.0+0.03), fontsize = 20, fontweight='bold')
plt.axis('off')
plt.show()

**NB** => mostrare distribuzione per confrontare la frequenza di una parola nei tre momenti analizzati (prima della campagna, durante e post)